# Calibrate $\pi$ Pulses
*Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## Outline

This tutorial introduces how to calibrate a $\pi$ pulse by varying the amplitude of the drive pulse. The outline of this tutorial is as follows:
- Introduction
- Preparation
- Define the system Hamiltonian
- Sweep amplitudes
- Cosine regression
- Summary

## Introduction

Calibrating $\pi$ pulses is one of the most fundamental operations in quantum computing because one of the most fundamental gates, the X gate, requires a $\pi$ pulse input onto the X channel. Further, it also serves an important role in calibrating actual hardware.

This tutorial will demonstrate how to calibrate a $\pi$ pulse using Quanlse.

## Preparation

After successfully installing Quanlse, you could run the program below to calibrate the $\pi$ pulses. To run this particular tutorial, you would need to import the following packages from Quanlse and other commonly-used Python libraries:

In [ ]:
# Import the Hamiltonian module
from Quanlse.QHamiltonian import QHamiltonian as QHam 

# Import related packages
from Quanlse.QOperator import duff, driveX
from Quanlse.QWaveform import gaussian, QJob, QJobList

# Import simulator interface for Quanlse Cloud Service
from Quanlse.remoteSimulator import remoteSimulatorRunHamiltonian as runHamiltonian

# Import numpy
from numpy import linspace, pi, dot, array, cos

# Import matplotlib
import matplotlib.pyplot as plt

# Import curve_fit function from scipy
from scipy.optimize import curve_fit

## Define the system Hamiltonian

In the field of quantum control, it is a common practice to describe a quantum system with its Hamiltonian. Generally, a system Hamiltonian consists of two terms, the time-independent and the time-dependent terms:

$$
\hat{H}_{\rm total}(t) = \hat{H}_{\rm drift} + \hat{H}_{\rm ctrl }(t) .
$$


We start with a single-qubit system with three energy levels. The system Hamiltonian can be written as:

$$
\hat{H} = \alpha_q \hat{a}^{\dagger}\hat{a}^{\dagger}\hat{a}\hat{a} + \frac{1}{2} c(t) \cos(\phi) (\hat{a}+\hat{a}^{\dagger}).
$$

Here, $\alpha_q$ is the anharmonicity between the two lowest transition energy levels. $c(t)$ indicates the pulse envelope function, and $\phi$ is the pulse phase. $\hat{a}^{\dagger}=|1\rangle\langle 0|+\sqrt{2}|2\rangle\langle 1|$ and $\hat{a}=|0\rangle\langle 1|+\sqrt{2}|1\rangle\langle 2|$ are respectively the creation and annihilation operators.

Users could easily create the Hamiltonian object for this system using the `QHamiltonian` module in Quanlse:

In [ ]:
ham = QHam(subSysNum=1, sysLevel=3, dt=0.2)

The above `QHam()` function returns a Hamiltonian, including the number of qubits and their energy levels, and sampling period.

Then we could add terms to this Hamiltonian. For example, the function `addDrift()` adds drift operators to the Hamiltonian. It basically requires a QHam object `ham`, the accordingly operators (we have provided the `QOperator` module which includes many commonly-used operators), the qubit(s) index(es) which the operators are acting upon, and the amplitude `coef`:

In [ ]:
alphaQ = - 0.22 * (2 * pi)  # unit is GHz
ham.addDrift(duff, 0, coef=alphaQ)

Here we could conveniently use `QOperator`'s method `duff()` to define the $n$-dimensional $\hat{a}^{\dagger}\hat{a}^{\dagger}\hat{a}\hat{a}$.

Then, the user could use the `print()` function to display the properties of this Hamiltonian:

In [ ]:
print(ham)

Next, we add the control terms to the Hamiltonian we created before by `addWave()`. Compared with the previous version of Quanlse, we have updated the strategy of adding control pulses by adding the operator and its accordingly waveform simultaneously. Here, we need to add the effective pulse:

$$
c(t) = A e^{-(t-\tau)^2/2\sigma^2}.
$$

The `addWave()` function takes the control term operator `driveX()`, the target qubit's index, and its waveform (Quanlse supports multiple waveforms' definitions) with parameters needed to define the wave.

In [ ]:
ham.appendWave(driveX, 0, gaussian(t=20, a=1.0, tau=10.0, sigma=3.0))

So far, we have just defined a complete quantum system and the parameters regarding controlling the system. Then, we can plot the pulse jobs by `plot()` the QJob class of our Hamiltonian. The function also includes an optional bool parameter `dark`, which enables a dark-themed mode. Moreover, the user can use the `color` parameter to specify colors for individual pulses (the colors will repeat if there are more pulses than colors).

In [ ]:
ham.job.plot(dark=True, color=['mint'])

Then we can use the `simulate()` function to simulate the evolution and obtain some important information, including the unitary matrix of the system evolution.

In [ ]:
result = ham.simulate(recordEvolution=False)
result.result

## Sweep amplitudes

With fixed pulse duration $t_g$, we can sweep the pulse's amplitudes $a$, and find the amplitude $a_{\pi}$ of the according $\pi$ pulse.

We first create a list of 200 points between -1 and 1, representing the pulse's amplitudes. 

In [ ]:
# Initilize the pulse's amplitudes
aList = linspace(-1.0, 1.0, 200)

Then, we can obtain the according population for each state by simulating the evolution of the Hamiltonian defined in the previous section. The calculation usually takes a long time to process on local devices; however, we provide a cloud computing service that could speed up this process significantly. To use Quanlse Cloud Service, the users can get a token from http://quantum-hub.baidu.com and submit the job onto Quanlse's server. Note that Quanlse supports the submission of batches of jobs, which could further optimize resource allocation.

In [ ]:
# Calibrate a Pi Pulse
jobList = ham.createJobList()
for a in aList:
    # Configure pulse parameters
    job = jobList.createJob()
    job.appendWave(driveX, 0, gaussian(20, a=a, tau=10, sigma=3))
    jobList.addJob(job)

# Import Define class and set the token
# Please visit http://quantum-hub.baidu.com
from Quanlse import Define
Define.hubToken = ""

# Submit batch jobs to Quanlse Cloud Service
resultList = runHamiltonian(ham, jobList=jobList)

# Calculate populations
pop0List = []
pop1List = []
pop2List = []
for result in resultList:
    finalState = dot(result["unitary"], array([1, 0, 0], dtype=complex))
    pop0List.append(abs(finalState[0])**2)
    pop1List.append(abs(finalState[1])**2)
    pop2List.append(abs(finalState[2])**2)

# Plot graph
plt.plot(aList, pop0List, label="Ground state")
plt.plot(aList, pop1List, label="1st excited state")
plt.plot(aList, pop2List, label="2nd excited state")
plt.xlabel("Amplitude")
plt.ylabel("Population of different states")
plt.legend()
plt.show()

## Cosine regression

Now, we have a series of discrete points, and we need to fit those points with a cosine function to find the amplitude of the $\pi$ pulse. To fit the resulting $|0\rangle$ population, we use the `optimize.curve_fit()` method in `Scipy`. We first define the following function:

In [ ]:
def fit_function(xValues, yValues, initParams):
    def fit_func(x, A, B, period, phi):
        return A * cos(2 * pi * x / period - phi) + B
    fitParams, _ = curve_fit(fit_func, xValues, yValues, initParams, bounds=(0, [2.0, 2.0, 2.0, 2.0]), method='dogbox')
    yFit = fit_func(xValues, *fitParams)
    return fitParams, yFit

Then we run the regression function to obtain the result:

In [ ]:
fitParams, yFit = fit_function(aList, pop0List, [0.5, 0.5, 0.8, 0])

# Plot graph
plt.scatter(aList, pop0List, label="Samples")
plt.plot(aList, yFit, color="red", label="Fit curve")
plt.xlabel("Amplitude")
plt.ylabel("Population of ground state")
plt.legend()
plt.show()
print(f"Period is {fitParams[2]}")
print(f"Pi pulse amplitude is {fitParams[2] / 2}")

By the cosine regression, we have identified the corresponding amplitude of the $\pi$ pulse is around 0.42.

## Summary

After reading this tutorial on calibrating  𝜋  pulses, users could follow this link [tutorial-calibrate-pi-pulses.ipynb](https://github.com/baidu/Quanlse/blob/main/Tutorial/EN/tutorial-pi-pulse.ipynb) to the GitHub page of this Jupyter Notebook document and obtain the relevant codes for themselves. In addition, the users are encouraged to try parameter values different from this tutorial to understand Quanlse better.